# Niloufar Baba Ahmadi 610398103 Assignment 2

- The 8-puzzle problem is a classic problem in artificial intelligence, where the goal is to find a sequence of moves to transform a given initial state of the puzzle into a goal state.

- A transition model defines how to move from one state of the puzzle to another by making a move. In the 8-puzzle problem, a state is defined as an arrangement of the numbers 1 to 8 and a blank space (0) in a 3x3 grid. The transition model defines how to move the blank space (0) up, down, left, or right.

- The path cost is a measure of the cost of reaching the current state from the initial state. In the 8-puzzle problem, each move has the same cost, so the path cost is simply the number of moves made so far.

- In the 8-puzzle problem, the goal state is a fixed arrangement of the numbers 1 to 8 and the blank space in a 3x3 grid.

- Actions are the possible moves that can be made from a state. In the 8-puzzle problem, the actions are up, down, left, and right.

- States are the different arrangements of the puzzle that can be reached through the actions.

- The best heuristic function for the 8-puzzle problem is the manhattan distance. This function calculates the total number of moves it would take to get each tile to its final position. The manhattan distance is admissible, meaning it never overestimates the cost to reach the goal, and it is consistent, meaning that the cost of moving from one state to another always satisfies the triangle inequality. This makes the manhattan distance an ideal heuristic function for the 8-puzzle problem and A* search algorithm.

### Imports

In [1]:
import tracemalloc
import time
import sys
from collections import deque
import heapq
import pandas as pd

### Find Zero

This function finds the location of 0 by iterating through the puzzel

In [6]:
def find_zero(state):

    for i in range(9):

        if state[i] == 0:

            return (i//3, i%3)
            
    return None

### Valid Moves

This function is implementing the logic for finding valid moves in a sliding puzzle game. The puzzle is represented by a state, which is a 1-dimensional list of 9 integers. The function takes the state and the current position of the empty square (represented by 0) as input and returns a list of valid moves and their resulting states.

The function first initializes an empty list called moves. It then checks if the empty square is not in the first row (zero_row > 0), and if so, it creates a copy of the current state (new_state = state.copy()) and swaps the empty square with the square above it. The move and resulting state are then appended to the moves list.

The function does similar checks for the other three directions (down, left, and right), and each time it finds a valid move, it creates a copy of the current state, makes the move, and appends the move and resulting state to the moves list.

Finally, the function returns the moves list, which contains all the valid moves and their resulting states.

In [7]:
def valid_moves(state, zero_row, zero_col):
    moves = []
    if zero_row > 0:
        new_state = state.copy()
        new_state[zero_row*3 + zero_col], new_state[(zero_row-1)*3 + zero_col] = new_state[(zero_row-1)*3 + zero_col], new_state[zero_row*3 + zero_col]
        moves.append(("up", new_state))
    if zero_row < 2:
        new_state = state.copy()
        new_state[zero_row*3 + zero_col], new_state[(zero_row+1)*3 + zero_col] = new_state[(zero_row+1)*3 + zero_col], new_state[zero_row*3 + zero_col]
        moves.append(("down", new_state))
    if zero_col > 0:
        new_state = state.copy()
        new_state[zero_row*3 + zero_col], new_state[zero_row*3 + zero_col-1] = new_state[zero_row*3 + zero_col-1], new_state[zero_row*3 + zero_col]
        moves.append(("left", new_state))
    if zero_col < 2:
        new_state = state.copy()
        new_state[zero_row*3 + zero_col], new_state[zero_row*3 + zero_col+1] = new_state[zero_row*3 + zero_col+1], new_state[zero_row*3 + zero_col]
        moves.append(("right", new_state))
    return moves

### DFS

DFS (Depth-First Search) is an algorithm for traversing and searching a graph or tree data structure. It starts from a source node and explores as far as possible along each branch before backtracking. The algorithm continues this process until all the nodes have been visited or a specific node has been found. DFS can be implemented using a stack data structure, where the nodes to be explored are pushed onto the stack and the most recently added node is explored first.

In [8]:
def dfs(start, goal):
    stack = [(start, [])]
    visited = set(tuple(start))
    while stack:
        state, path = stack.pop()
        if state == goal:
            return path
        zero_row, zero_col = find_zero(state)
        for move, new_state in valid_moves(state, zero_row, zero_col):
            if tuple(new_state) not in visited:
                visited.add(tuple(new_state))
                stack.append((new_state, path + [(move, new_state)]))
    return None

### BFS

BFS (Breadth-First Search) is a graph traversal algorithm that visits all the vertices of a graph in breadth-first order, i.e. it visits all the vertices at the same level before visiting vertices at the next level. It uses a queue data structure to store the vertices to be visited next and visits each vertex only once. 

In [9]:
def bfs(start, goal):
    queue = deque([(start, [])])
    visited = set(tuple(start))
    while queue:
        state, path = queue.popleft()
        if state == goal:
            return path
        zero_row, zero_col = find_zero(state)
        for move, new_state in valid_moves(state, zero_row, zero_col):
            if tuple(new_state) not in visited:
                visited.add(tuple(new_state))
                queue.append((new_state, path + [(move, new_state)]))
    return None

### IDS

IDS (Iterative Deepening Search) is a search algorithm that combines the advantages of breadth-first search (BFS) and depth-first search (DFS). The algorithm starts by exploring the shallowest nodes in the tree first and incrementally increasing the depth limit of the search, repeating the search process until a goal node is found. This way, IDS ensures that the optimal solution is found, while avoiding the memory overhead of BFS and the infinite looping of DFS. 

In [10]:
def dfs_limited(start, goal, limit, path):
    if start == goal:
        return path
    if limit == 0:
        return None
    zero_row, zero_col = find_zero(start)
    for action, new_state in valid_moves(start, zero_row, zero_col):
        result = dfs_limited(new_state, goal, limit-1, path + [(action, new_state)])
        if result is not None:
            return result
    return None

def ids(start, goal):
    for depth in range(100):
        result = dfs_limited(start, goal, depth, [(None, start)])
        if result is not None:
            return result
    return None

### UCS

Uniform Cost Search (UCS) is a search algorithm used for finding the shortest path from a source node to a goal node in a weighted graph. It works by exploring all the nodes in the graph, starting from the source node, and expanding the node with the lowest cost first. The cost of reaching a node is the sum of the cost of reaching its parent node and the cost of moving from the parent node to the current node. The algorithm continues this process until the goal node is reached or all nodes have been explored. 

In [11]:
def ucs(start, goal):
    queue = [(0, start, [])]
    visited = set(tuple(start))
    while queue:
        cost, state, path = heapq.heappop(queue)
        if state == goal:
            return path
        zero_row, zero_col = find_zero(state)
        for move, new_state in valid_moves(state, zero_row, zero_col):
            if tuple(new_state) not in visited:
                visited.add(tuple(new_state))
                heapq.heappush(queue, (cost + 1, new_state, path + [(move, new_state)]))
    return None

### A⋆

A⋆ is a search algorithm that combines the strengths of both Breadth-First Search (BFS) and Greedy Best-First Search algorithms. It finds the path with the lowest cost (often represented by a heuristic) from the start node to the goal node. The algorithm works by maintaining two lists: an open list of nodes to visit, and a closed list of nodes that have been visited. At each step, the algorithm selects the node with the lowest estimated cost from the open list, and adds its neighbors to the open list. The cost estimate is a combination of the actual cost from the start node to the current node, and an estimated cost from the current node to the goal node (the heuristic). The algorithm continues until the goal node is reached or there are no more nodes in the open list. 

In [12]:
def manhattan_distance(state, goal):
    distance = 0
    for i in range(9):
        tile = state[i]
        if tile == 0:
            continue
        row, col = i // 3, i % 3
        goal_row, goal_col = (tile - 1) // 3, (tile - 1) % 3
        distance += abs(row - goal_row) + abs(col - goal_col)
    return distance

def a_star(start, goal):
    heap = [(manhattan_distance(start, goal), 0, start, [])]
    visited = set(tuple(start))
    while heap:
        _, cost, state, path = heapq.heappop(heap)
        if state == goal:
            return path
        zero_row, zero_col = find_zero(state)
        for move, new_state in valid_moves(state, zero_row, zero_col):
            if tuple(new_state) not in visited:
                visited.add(tuple(new_state))
                new_path = path.copy()
                new_path.append((move, new_state))
                new_cost = cost + 1
                heapq.heappush(heap, (manhattan_distance(new_state, goal) + new_cost, new_cost, new_state, new_path))
    return None

### Creating the output files

Here we make a file which consistes of Action sequence and resource allocations (time and memory) for each puzzle solved by a certain search algorithm.

In [24]:
import time
import tracemalloc

def main():
    examples = [[1,2,3,0,7,6,5,4,8],
                [0,4,1,2,5,3,7,8,6],
                [4,1,3,0,2,6,7,5,8],
                [1,2,3,0,4,8,7,6,5],
                [1,2,0,4,8,3,7,6,5],
                [1,0,2,4,6,3,7,5,8],
                [0,1,2,4,5,3,7,8,6],
                [1,2,3,0,4,5,7,8,6],
                [1,2,3,4,0,5,7,8,6],
                [1,2,3,4,5,0,7,8,6],
                [0,1,3,4,2,5,7,8,6],
                [2,3,5,1,0,4,7,8,6],
                [1,6,2,5,3,0,4,7,8],
                [1,8,2,0,4,3,7,6,5],
                [2,5,3,4,1,6,0,7,8],
                [1,2,3,4,6,8,7,5,0],
                [1,6,2,5,7,3,0,4,8],
                [0,4,1,5,3,2,7,8,6],
                [0,5,2,1,8,3,4,7,6],
                [1,2,3,0,4,6,7,5,8]
                ]

    goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    search_functions = [("DFS"), dfs, ("BFS", bfs), ("IDS", ids), ("UCS", ucs), ("A*", a_star)]

    for search_func in search_functions:
        with open("output_easy.txt", "a") as file:
            for example in examples:
                func_name, func = search_func
                file.write("{}\n".format(func_name))
                tracemalloc.start()
                start = time.time()
                path = func(example, goal)
                current, peak = tracemalloc.get_traced_memory()
                end = time.time()
                if path is not None:
                    file.write("Number of moves: {}\n".format(len(path)))
                    file.write("Path to goal: {}\n".format(path))
                    file.write("Time taken: {}\n".format(end - start))
                    file.write("Peak memory usage: {}MB\n".format(peak / 10**6))
                else:
                    file.write("No solution found\n")

if __name__ == "__main__":
    main()


### Create Excel file

To better understand the efficiency of each algorithm, we create a CSV file to compare the time and memory used by each algorithm.

In [ ]:
def main():
    examples = [[1,2,3,0,7,6,5,4,8],
                [0,4,1,2,5,3,7,8,6],
                [4,1,3,0,2,6,7,5,8],
                [1,2,3,0,4,8,7,6,5],
                [1,2,0,4,8,3,7,6,5],
                [1,0,2,4,6,3,7,5,8],
                [0,1,2,4,5,3,7,8,6],
                [1,2,3,0,4,5,7,8,6],
                [1,2,3,4,0,5,7,8,6],
                [1,2,3,4,5,0,7,8,6],
                [0,1,3,4,2,5,7,8,6],
                [2,3,5,1,0,4,7,8,6],
                [1,6,2,5,3,0,4,7,8],
                [1,8,2,0,4,3,7,6,5],
                [2,5,3,4,1,6,0,7,8],
                [1,2,3,4,6,8,7,5,0],
                [1,6,2,5,7,3,0,4,8],
                [0,4,1,5,3,2,7,8,6],
                [0,5,2,1,8,3,4,7,6],
                [1,2,3,0,4,6,7,5,8]
                ]

    goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    data = {"dfs_time": [], "dfs_memory": [],"bfs_time": [], "bfs_memory": [], "ids_time": [], "ids_memory": [], "ucs_time": [], "ucs_memory": [], "A_time": [], "A_memory": []}
    
    for example in examples:

        tracemalloc.start()
        start_time = time.time()
        path = dfs(example, goal)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        data["dfs_time"].append(end_time - start_time)
        data["dfs_memory"].append(peak / 10**6)

        tracemalloc.start()
        start_time = time.time()
        path = bfs(example, goal)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        data["bfs_time"].append(end_time - start_time)
        data["bfs_memory"].append(peak / 10**6)

        tracemalloc.start()
        start_time = time.time()
        path = ids(example, goal)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        data["ids_time"].append(end_time - start_time)
        data["ids_memory"].append(peak / 10**6)

        tracemalloc.start()
        start_time = time.time()
        path = ucs(example, goal)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        data["ucs_time"].append(end_time - start_time)
        data["ucs_memory"].append(peak / 10**6)

        tracemalloc.start()
        start_time = time.time()
        path = a_star(example, goal)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        data["A_time"].append(end_time - start_time)
        data["A_memory"].append(peak / 10**6)

    df = pd.DataFrame(data)
    print(df)
    df.to_csv("results_easy.csv", index=False)

if __name__ == "__main__":
    main()


### Conclusions 

- BFS (Breadth-first search) is guaranteed to find the optimal solution if it exists, but its time complexity can be really high.

- DFS (Depth-first search) is less likely to find the optimal solution and is also slow and I just hated DFS this whole time. I mean, I can't even use colab to run this one! thats how awful it is!! 

- IDS (Iterative deepening depth-first search) is a combination of DFS and BFS, and it is often used for puzzles like the 8 puzzle because it balances the time complexity and the solution quality.

- UCS (Uniform cost search) and A* (A-star) algorithms are similar to BFS, but they use a heuristic function to guide the search. They are more efficient than BFS and IDS. Also, often find the optimal solution in a much shorter time. However, the choice of heuristic function can greatly impact the performance of these algorithms, so it's important to choose a good heuristic for the particular problem.

- Overall, A* is the best algorithm for solving these puzzles because of its efficiency and ability to find the optimal solution in the shortest time compared to other algorithms.



### Average resources used for easy puzzles

In [5]:
df_easy = pd.read_csv('/content/results_easy.csv')
df_easy.describe()

,dfs_time,dfs_memory,bfs_time,bfs_memory,ids_time,ids_memory,ucs_time,ucs_memory,A_time,A_memory
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,24.084736,9532.169655,0.010220,9532.169655,0.014943,9532.169655,0.000748,9532.169655,0.000127,9532.169655
std,23.388239,817.294250,0.012005,817.294250,0.043788,817.294250,0.001197,817.294250,0.000091,817.294250
min,0.000032,8678.533657,0.000075,8678.533657,0.000034,8678.533657,0.000066,8678.533657,0.000045,8678.533657
25%,0.007341,8678.533657,0.000219,8678.533657,0.000209,8678.533657,0.000142,8678.533657,0.000072,8678.533657
50%,22.151939,10243.246234,0.000462,10243.246234,0.000556,10243.246234,0.000318,10243.246234,0.000090,10243.246234
75%,47.366122,10243.590170,0.020259,10243.590170,0.003565,10243.590170,0.000787,10243.590170,0.000160,10243.590170
max,54.607463,10243.591002,0.029623,10243.591002,0.146722,10243.591002,0.004179,10243.591002,0.000334,10243.591002


### Average resources used for medium puzzles

In [6]:
df_medium = pd.read_csv('/content/results_medium.csv')
df_medium.describe()

,dfs_time,dfs_memory,bfs_time,bfs_memory,ids_time,ids_memory,ucs_time,ucs_memory,A_time,A_memory
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,21.929235,10243.591002,0.097476,10243.591002,37.691197,10243.591002,0.072943,10243.591002,0.001057,10243.591002
std,20.697269,0.000000,0.065989,0.000000,49.871948,0.000000,0.066562,0.000000,0.000788,0.000000
min,2.389755,10243.591002,0.014506,10243.591002,0.394014,10243.591002,0.008256,10243.591002,0.000275,10243.591002
25%,8.361347,10243.591002,0.042707,10243.591002,1.585443,10243.591002,0.021394,10243.591002,0.000503,10243.591002
50%,17.138896,10243.591002,0.110675,10243.591002,12.307280,10243.591002,0.058350,10243.591002,0.000791,10243.591002
75%,25.086261,10243.591002,0.130880,10243.591002,63.924003,10243.591002,0.110021,10243.591002,0.001506,10243.591002
max,62.781487,10243.591002,0.205054,10243.591002,137.001151,10243.591002,0.191665,10243.591002,0.002380,10243.591002


### Average resources used for hard puzzles

In [7]:
df_hard = pd.read_csv('/content/results_hard.csv')
df_hard.describe()

,dfs_time,dfs_memory,bfs_time,bfs_memory,ucs_time,ucs_memory,A_time,A_memory
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,23.864507,10243.591002,0.976281,10243.591002,1.047817,10243.591002,0.038775,10243.591002
std,17.801393,0.000000,0.644054,0.000000,0.716817,0.000000,0.043427,0.000000
min,4.307247,10243.591002,0.493921,10243.591002,0.527705,10243.591002,0.008718,10243.591002
25%,11.138389,10243.591002,0.576041,10243.591002,0.528176,10243.591002,0.010086,10243.591002
50%,25.092191,10243.591002,0.751907,10243.591002,0.807994,10243.591002,0.022430,10243.591002
75%,37.818309,10243.591002,1.152147,10243.591002,1.327635,10243.591002,0.051119,10243.591002
max,40.966400,10243.591002,1.907388,10243.591002,2.047573,10243.591002,0.101523,10243.591002
